In [1]:
import numpy as np
import pandas as pd
from math import sin, cos

In [2]:
# plant the seed for random variable
np.random.seed(20)

# assume that image is HD
h, w = (1080, 1920)

# assume the size of bbox
min_offset = 50
max_offset = 300

# generate classes
classes = np.random.randint(0, 20, size=(10,))

# generate positions 
x_min = np.random.randint(0, w-max_offset, size=(10,))
x_max = np.random.randint(min_offset, max_offset, size=(10,)) + x_min
y_min = np.random.randint(0, h-max_offset, size=(10,))
y_max = np.random.randint(min_offset, max_offset, size=(10,)) + y_min

fake_input = pd.DataFrame({
    'classes': classes, 
    'x_min': x_min, 
    'x_max': x_max, 
    'y_min': y_min, 
    'y_max': y_max
})

print(fake_input)


   classes  x_min  x_max  y_min  y_max
0        3    765    841    441    534
1       15    742    933    478    672
2        9   1214   1467    288    356
3       11   1168   1404    778    907
4        7    617    692    774   1042
5        2    775    956    274    511
6        0    485    737    259    508
7        8   1279   1532    509    634
8       19    994   1105    401    569
9       16   1030   1285    760    967


Formular 1: <font size="5"> $f_m = 1024^{\frac{2m}{d_model}}$</font>



Formular 2:  <font size="5"> $e(t) = [sin(\frac{t}{f_1}), cos(\frac{t}{f_1}), ... , sin(\frac{t}{f_{\frac{d_model}{2}}}), cos(\frac{t}{f_{\frac{d_model}{2}}})] $ </font>

In [3]:
# f value for one note with Hari's formula
def fValueOneNode(m, d):
    '''
    Input:
    + m: the input value for formula 1
    + d: dimension of embedded vector
    Output:
    + f value of 1 node
    '''
    return 1024*(2*m/d)

# one pair value
def EValue(t, f):
    '''
    Input:
    + t and f are the variable for formula 2
    Output:
    + e value for 1 node t
    '''
    meat = (float)(t/f)
    return [sin(meat), cos(meat)]

# generate embedded vector function
def embedingVector(max_index, dim):
    '''
    Input:
    + max_index: number of buttons in image
    + dim: dimension of embedded vector
    Output:
    + a vector with len dim//2, each node is e(t)
    '''
    f_value =  list(map(fValueOneNode, 
                        range(1, dim//2+1), 
                        [dim]*max_index))
    buck_of_vectors = []
    for t in range(1, max_index+1):
        buck_of_vectors.append(list(map(EValue, 
                                        [t]*(dim//2), 
                                        f_value)))
    result = np.array(buck_of_vectors)
    return result.reshape(result.shape[0],-1)


def main(df, dimension):
    '''
    Input: 
    + df: dataframe contain classes and positions of buttons
    + dimension: dimension of embedded vector
    Output:
    + A numpy array with shape (4, max_index, dim), with 4 in order is x_min, x_max, y_min, y_max
    each one contain max_index embedded vectors
    '''
    max_index = len(df.classes)
    vector = embedingVector(max_index, dimension)
    x_min = vector[np.argsort(np.array(df.x_min))]
    x_max = vector[np.argsort(~np.array(df.x_max))]
    y_min = vector[np.argsort(np.array(df.y_min))]
    y_max = vector[np.argsort(~np.array(df.y_max))]
    result = np.array([x_min, x_max, y_min, y_max])
    return result

    

In [12]:
embedded_vectors = main(fake_input, 16)
embedded_vectors

array([[[5.46602449e-02, 9.98505011e-01, 2.73403427e-02, 9.99626183e-01,
         1.82281571e-02, 9.99833853e-01, 1.36714491e-02, 9.99906541e-01,
         1.09372819e-02, 9.99940186e-01, 9.11445713e-03, 9.99958462e-01,
         7.81242053e-03, 9.99969483e-01, 6.83588426e-03, 9.99976635e-01],
        [3.90525667e-02, 9.99237158e-01, 1.95300083e-02, 9.99809271e-01,
         1.30204654e-02, 9.99915230e-01, 9.76546978e-03, 9.99952317e-01,
         7.81242053e-03, 9.99969483e-01, 6.51037068e-03, 9.99978807e-01,
         5.58032818e-03, 9.99984430e-01, 4.88279310e-03, 9.99988079e-01],
        [1.56243642e-02, 9.99877932e-01, 7.81242053e-03, 9.99969483e-01,
         5.20830979e-03, 9.99986437e-01, 3.90624007e-03, 9.99992371e-01,
         3.12499491e-03, 9.99995117e-01, 2.60416372e-03, 9.99996609e-01,
         2.23214100e-03, 9.99997509e-01, 1.95312376e-03, 9.99998093e-01],
        [7.81242053e-03, 9.99969483e-01, 3.90624007e-03, 9.99992371e-01,
         2.60416372e-03, 9.99996609e-01, 1.95312

In [7]:
fake_input["x_min"].sort_values()

6     485
4     617
1     742
0     765
5     775
8     994
9    1030
3    1168
2    1214
7    1279
Name: x_min, dtype: int32

In [19]:
np.argsort(embedded_vectors[0][:][:])

array([14, 12, 10,  8,  6,  4,  2,  0,  1,  3,  5,  7,  9, 11, 13, 15],
      dtype=int64)

In [21]:
embedded_vectors[0][:][:]

array([[5.46602449e-02, 9.98505011e-01, 2.73403427e-02, 9.99626183e-01,
        1.82281571e-02, 9.99833853e-01, 1.36714491e-02, 9.99906541e-01,
        1.09372819e-02, 9.99940186e-01, 9.11445713e-03, 9.99958462e-01,
        7.81242053e-03, 9.99969483e-01, 6.83588426e-03, 9.99976635e-01],
       [3.90525667e-02, 9.99237158e-01, 1.95300083e-02, 9.99809271e-01,
        1.30204654e-02, 9.99915230e-01, 9.76546978e-03, 9.99952317e-01,
        7.81242053e-03, 9.99969483e-01, 6.51037068e-03, 9.99978807e-01,
        5.58032818e-03, 9.99984430e-01, 4.88279310e-03, 9.99988079e-01],
       [1.56243642e-02, 9.99877932e-01, 7.81242053e-03, 9.99969483e-01,
        5.20830979e-03, 9.99986437e-01, 3.90624007e-03, 9.99992371e-01,
        3.12499491e-03, 9.99995117e-01, 2.60416372e-03, 9.99996609e-01,
        2.23214100e-03, 9.99997509e-01, 1.95312376e-03, 9.99998093e-01],
       [7.81242053e-03, 9.99969483e-01, 3.90624007e-03, 9.99992371e-01,
        2.60416372e-03, 9.99996609e-01, 1.95312376e-03, 9.999